# XtEHR Logical model processing

* ophalen XtEHR Logical Models
* consistente layout met zibs
* klaar voor een mapping - zie mapping Notebook

## Relevante kern content

NL kernset:
- Behandelaanwijzing

ePS required:
- Medicatie
- Diagnose/ Probleemlijst(actief)
- Verrichtingen(<6mnd, ≥ 6mnd)
- Allergieën en intoleranties + - - 
- Alerts
- Hulpmiddel
- Patiënt

ePS recommended + optional:
- Vaccinations
- Implants
- Treatment restrictions
- Vital signs
- History of illness
- Pregnancy
- Social history
- Functional status
- Results
- Plan of Care
- Encounters

# Imports

In [2]:
import pandas as pd
import requests
import json
import os
from io import StringIO

# Download JSON

* Dit hoeft maar een keer te gebeuren, bij nieuwe mappings download de JSON die nodig is. 
* Kan ook door directe download 'raw' JSON.
* Dit zijn de XtEHR Logical Models

In [10]:
xtehrlist = ['AlertFlag', 'AllergyIntolerance', 'Condition', 'Device','DeviceUse', 'Medication', 'Organization', 'Patient', 'Procedure']
xtehrbase = 'https://build.fhir.org/ig/Xt-EHR/xt-ehr-common/StructureDefinition-EHDS'
# res = 'AlertFlag'
if (not os.path.exists('xtehr')):
    os.makedirs('xtehr')
for res in xtehrlist:
    print(res)
    url = xtehrbase + res + '.json'
    response = requests.get(url)
    jsondata = response.json()
    with open(os.path.join('xtehr', 'downloads', res + '.json'), 'w') as jsonfile:
        json.dump(jsondata, jsonfile, indent=4)

AlertFlag
AllergyIntolerance
Condition
Device
DeviceUse
Medication
Organization
Patient
Procedure


# Kies en lees het te verwerken Logical Model

* Toon en verifieer dat de uitvoer er is

In [11]:
base = 'xtehr'
file = 'DeviceUse'
url = os.path.join(base, 'downloads', file + '.json')
with open(url) as jsonfile:
    xlm = json.load(jsonfile)['snapshot']['element']
dfx = pd.DataFrame(xlm)
dfx

,id,path,short,definition,min,max,base,isModifier,mapping,type,binding
0,EHDSDeviceUse,EHDSDeviceUse,Device use model,EHDS refined base model for Device Use,0,*,"{'path': 'Base', 'min': 0, 'max': '*'}",False,"[{'identity': 'rim', 'map': 'n/a'}]",NaN,NaN
1,EHDSDeviceUse.identifier,EHDSDeviceUse.identifier,An identifier for this statement.,An identifier for this statement.,0,*,"{'path': 'EHDSDeviceUse.identifier', 'min': 0,...",NaN,NaN,[{'code': 'Identifier'}],NaN
2,EHDSDeviceUse.subject,EHDSDeviceUse.subject,The patient using the device.,The patient using the device.,1,1,"{'path': 'EHDSDeviceUse.subject', 'min': 1, 'm...",NaN,NaN,[{'code': 'https://www.xt-ehr.eu/specification...,NaN
3,EHDSDeviceUse.status,EHDSDeviceUse.status,Current status of the Device Usage.,Current status of the Device Usage.,0,1,"{'path': 'EHDSDeviceUse.status', 'min': 0, 'ma...",NaN,NaN,[{'code': 'CodeableConcept'}],"{'strength': 'preferred', 'description': 'HL7 ..."
4,EHDSDeviceUse.implantDate,EHDSDeviceUse.implantDate,Date when procedure was performed.,Date when procedure was performed.,0,1,"{'path': 'EHDSDeviceUse.implantDate', 'min': 0...",NaN,NaN,[{'code': 'dateTime'}],NaN
5,EHDSDeviceUse.endDate,EHDSDeviceUse.endDate,Date when the device was explanted from the pa...,Date when the device was explanted from the pa...,0,1,"{'path': 'EHDSDeviceUse.endDate', 'min': 0, 'm...",NaN,NaN,[{'code': 'dateTime'}],NaN
6,EHDSDeviceUse.device,EHDSDeviceUse.device,The details of the device used.,The details of the device used.,1,1,"{'path': 'EHDSDeviceUse.device', 'min': 1, 'ma...",NaN,NaN,[{'code': 'https://www.xt-ehr.eu/specification...,NaN
7,EHDSDeviceUse.bodySite,EHDSDeviceUse.bodySite,Anatomical location of the device. May include...,Anatomical location of the device. May include...,0,1,"{'path': 'EHDSDeviceUse.bodySite', 'min': 0, '...",NaN,NaN,[{'code': 'CodeableConcept'}],NaN
8,EHDSDeviceUse.note,EHDSDeviceUse.note,Note about the device statement that were not ...,Note about the device statement that were not ...,0,*,"{'path': 'EHDSDeviceUse.note', 'min': 0, 'max'...",NaN,NaN,[{'code': 'string'}],NaN
9,EHDSDeviceUse.recorded,EHDSDeviceUse.recorded,Date and time at which the statement was made/...,Date and time at which the statement was made/...,0,1,"{'path': 'EHDSDeviceUse.recorded', 'min': 0, '...",NaN,NaN,[{'code': 'dateTime'}],NaN


# Opschonen uitvoer zodat zib en xtehr consistent zijn

* een card. veld met n..m
* type (is een JSON object) omzetten naar tekst
* alleen nodige kolommen houden

In [12]:
dfx.insert(2, 'card.', None)
dfx['card.'] = dfx['min'].astype(str) + '..' + dfx['max'].astype(str)
for i, row in dfx.iterrows():
    if 'type' in row:
        # row['type'][0]['code']
        try:
            dfx.at[i, 'type'] = row['type'][0]['code'].split('/')[-1]
        except TypeError:
            pass
dfx= dfx[['id', 'path', 'short', 'definition', 'type', 'card.', 'binding']]
dfx

,id,path,short,definition,type,card.,binding
0,EHDSDeviceUse,EHDSDeviceUse,Device use model,EHDS refined base model for Device Use,NaN,0..*,NaN
1,EHDSDeviceUse.identifier,EHDSDeviceUse.identifier,An identifier for this statement.,An identifier for this statement.,Identifier,0..*,NaN
2,EHDSDeviceUse.subject,EHDSDeviceUse.subject,The patient using the device.,The patient using the device.,EHDSPatient,1..1,NaN
3,EHDSDeviceUse.status,EHDSDeviceUse.status,Current status of the Device Usage.,Current status of the Device Usage.,CodeableConcept,0..1,"{'strength': 'preferred', 'description': 'HL7 ..."
4,EHDSDeviceUse.implantDate,EHDSDeviceUse.implantDate,Date when procedure was performed.,Date when procedure was performed.,dateTime,0..1,NaN
5,EHDSDeviceUse.endDate,EHDSDeviceUse.endDate,Date when the device was explanted from the pa...,Date when the device was explanted from the pa...,dateTime,0..1,NaN
6,EHDSDeviceUse.device,EHDSDeviceUse.device,The details of the device used.,The details of the device used.,EHDSDevice,1..1,NaN
7,EHDSDeviceUse.bodySite,EHDSDeviceUse.bodySite,Anatomical location of the device. May include...,Anatomical location of the device. May include...,CodeableConcept,0..1,NaN
8,EHDSDeviceUse.note,EHDSDeviceUse.note,Note about the device statement that were not ...,Note about the device statement that were not ...,string,0..*,NaN
9,EHDSDeviceUse.recorded,EHDSDeviceUse.recorded,Date and time at which the statement was made/...,Date and time at which the statement was made/...,dateTime,0..1,NaN


# Opslaan in Excel

In [13]:
url = os.path.join(base, 'mappable', file + '.xlsx')
dfx.to_excel(url, index=False)

PermissionError: [Errno 13] Permission denied: 'xtehr\\mappable\\DeviceUse.xlsx'